In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import os

In [2]:
os.chdir('./data')
df_main = pd.read_csv('processed_data.csv')
df_main.set_index('timestamp', inplace=True)
df_main.index = pd.to_datetime(df_main.index)
df_main.columns

Index(['SP_load_MAW', 'DE_B09_MAW', 'DE_B20_MAW', 'SE_load_MAW', 'NE_B20_MAW',
       'NE_B18_MAW', 'NE_B19_MAW', 'DK_load_MAW', 'DE_B18_MAW', 'DE_B19_MAW',
       ...
       'PO_green_surplus_MAW', 'HU_green_MAW', 'HU_green_surplus_MAW',
       'IT_green_MAW', 'IT_green_surplus_MAW', 'UK_green_MAW',
       'UK_green_surplus_MAW', 'max_surplus_country_name',
       'max_surplus_country_code', 'max_surplus_country_code_next_hr'],
      dtype='object', length=135)

In [4]:
relevant_countries = ['DK', 'UK', 'SE', 'DE']
relevant_cols = [col for col in df_main.columns if ('green_surplus' in col and col.split('_')[0] in relevant_countries)]

# reverse order of relevant_cols
relevant_cols = relevant_cols[::-1]
df_green_surplus= df_main[relevant_cols]

print(len(df_green_surplus))
print(df_green_surplus.columns)

8761
Index(['UK_green_surplus_MAW', 'DK_green_surplus_MAW', 'SE_green_surplus_MAW',
       'DE_green_surplus_MAW'],
      dtype='object')


In [8]:
find_best_lags_bool = False

if (find_best_lags_bool == True) :
        
    p_vals = range(3,6)
    q_vals = range(3,6)

    all_res = []

    for idx, data_name in enumerate(relevant_cols) :

        print('= '*30)
        print(f"start {data_name} {idx+1}/{len(relevant_cols)}")

        # demean data
        mean = df_green_surplus[data_name].mean()
        data = df_green_surplus[data_name] - mean
        # set frequency to hourly
        data = data.asfreq('H')

        # Create dataframe to store results
        results = []

        # Loop through p and q values
        for p_val in p_vals:
            for q_val in q_vals:

                print(f"p={p_val}, q={q_val}")

                try:
                    # Fit ARIMA model
                    order = (p_val, 0, q_val)
                    model = ARIMA(data, order=order)
                    fit_model = model.fit(low_memory=True)

                    # Get BIC and AIC values
                    bic = fit_model.bic
                    aic = fit_model.aic

                    # Save results in the dataframe
                    results.append({'country' : data_name, 'mean' : mean,
                                    'p': p_val, 'q': q_val, 
                                    'BIC': bic, 'AIC': aic, 'IC_comb' : bic+aic})

                except Exception as e:
                    print(f"An error occurred: {e}")

        results_df = pd.DataFrame(results)
        print(results_df)

        # Find the best p and q based on lowest combined AIC and BIC
        best_params = results_df.loc[results_df['IC_comb'].idxmin()]
        print('-'*30)
        print(f"Best p = {best_params['p']}, best q = {best_params['q']}")
        print('-'*30)

        # save best params p and q in dataframe for each loop iteration
        all_res.append(best_params)

    all_res_df = pd.DataFrame(all_res)

else :

    # resort to historic best lags from test phase

    data = {
    'country': ['DE_green_surplus_MAW', 'SE_green_surplus_MAW', 'DK_green_surplus_MAW', 'UK_green_surplus_MAW'],
    'p': [3, 5, 5, 5],
    'q': [5, 3, 4, 4]
}

all_res_df = pd.DataFrame(data)

In [10]:
model_dict = {}
df_forecast = pd.DataFrame()
df_data = df_green_surplus[relevant_cols]
h = 24

for data_name in relevant_cols :

    # demean data
    mean = df_green_surplus[data_name].mean()
    data = df_green_surplus[data_name] - mean
    # set frequency to hourly
    data = data.asfreq('H')
    
    res_row = all_res_df[all_res_df['country'] == data_name]
    order = (res_row['p'], 0, res_row['q'])

    final_model = ARIMA(data, order=order)
    final_fit = final_model.fit()
    model_dict[data_name] = final_fit

    # In-sample predictions
    predictions = final_fit.predict()
    df_data[f"{data_name}_insamp_pred"] = predictions + mean

    # Make forecast for next h hours
    forecast = model_dict[data_name].forecast(steps=h)

    # Add forecast + mean to df_forecast
    mean = df_green_surplus[data_name].mean()
    df_forecast[data_name] = forecast + mean
    
    # Plot the original data, in sample predictions, and fcast
    # plt.figure(figsize=(20, 5))

    # plt.plot(data, label='Actual Data')
    # plt.plot(predictions, label='In-sample Predictions', color='red', lw=1, linestyle='--', alpha=0.5)
    # plt.plot(forecast, label='Forecast', color='green', lw=2, linestyle='-')

    # plt.title(f"{data_name} - ARMA({order}) In-sample Predictions")
    # plt.legend()
    # plt.show()

# calculate which fcast is highest and put in column 'max_surplus_country_name'
df_forecast['max_surplus_country_name'] = df_forecast.apply(lambda row: max(relevant_countries, key=lambda country: row[f"{country}_green_surplus_MAW"]), axis=1)

/Users/yanting/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/yanting/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/yanting/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


In [16]:
df_data['y_true'] = df_data.apply(lambda row: max(relevant_countries, key=lambda country: row[f"{country}_green_surplus_MAW"]), axis=1)
df_data['y_pred'] = df_data.apply(lambda row: max(relevant_countries, key=lambda country: row[f"{country}_green_surplus_MAW_insamp_pred"]), axis=1)

timestamp
2021-12-31 23:00:00+00:00    False
2022-01-01 00:00:00+00:00    False
2022-01-01 01:00:00+00:00    False
2022-01-01 02:00:00+00:00    False
2022-01-01 03:00:00+00:00    False
                             ...  
2022-12-31 19:00:00+00:00    False
2022-12-31 20:00:00+00:00    False
2022-12-31 21:00:00+00:00    False
2022-12-31 22:00:00+00:00    False
2022-12-31 23:00:00+00:00    False
Name: y_true, Length: 8761, dtype: bool

In [50]:
df_data[(df_data['y_true'] == 'DK') & (df_data['y_pred'] == 'DK')].head()

,UK_green_surplus_MAW,DK_green_surplus_MAW,SE_green_surplus_MAW,DE_green_surplus_MAW,UK_green_surplus_MAW_insamp_pred,DK_green_surplus_MAW_insamp_pred,SE_green_surplus_MAW_insamp_pred,DE_green_surplus_MAW_insamp_pred,y_true,y_pred
timestamp,,,,,,,,,,
2021-12-31 23:00:00+00:00,-1337.5,338.0,-4271.0,-5142.0,-2901.698836,-1042.941231,-3292.547606,-108753.946003,DK,DK
2022-01-01 00:00:00+00:00,-2452.5,387.0,-4224.0,-19968.0,-2799.823308,-736.134254,-3292.547606,-148187.980417,DK,DK
2022-01-01 01:00:00+00:00,-2256.5,183.0,-4234.0,-22014.0,-2878.981470,-780.455392,-3605.372914,-229281.197207,DK,DK
2022-01-01 02:00:00+00:00,-2152.0,-37.0,-4641.0,-25418.0,-2860.983468,-817.164969,-3935.316801,-20199.346332,DK,DK
2022-01-01 03:00:00+00:00,-1912.5,-192.0,-4617.0,-30005.0,-2197.242059,-441.708244,-3842.367135,-34494.332582,DK,DK


In [51]:
df_data['y_true'].tolist()

['DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'UK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'UK',
 'UK',
 'UK',
 'UK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',
 'DK',

In [70]:
from sklearn.metrics import classification_report

y_true = df_data['y_true']
y_pred = df_data['y_pred']
print(classification_report(y_true, y_pred, zero_division=0))

              precision    recall  f1-score   support

          DE       0.00      0.00      0.00        20
          DK       0.84      0.96      0.90      6852
          SE       0.74      0.44      0.55      1401
          UK       0.07      0.02      0.04       488

    accuracy                           0.82      8761
   macro avg       0.41      0.35      0.37      8761
weighted avg       0.78      0.82      0.79      8761



In [54]:
from sklearn.metrics import classification_report

y_true = df_data['y_true'].tolist()[:10]
y_pred = df_data['y_pred'].tolist()[:10]

print(y_true)
print(y_pred)

y_true = y_true
y_pred = y_pred
target_names = ['DK', 'UK']
print(classification_report(y_true, y_pred, target_names=target_names, zero_division=0))

              precision    recall  f1-score   support

          DK       0.90      1.00      0.95         9
          UK       0.00      0.00      0.00         1

    accuracy                           0.90        10
   macro avg       0.45      0.50      0.47        10
weighted avg       0.81      0.90      0.85        10

